In [1]:
from bs4 import BeautifulSoup
import json
import re

with open('Data/bookmarks_19.04.2024.html', 'r', encoding='utf-8') as f:
    bookmarksHtml = f.read()

In [2]:
soup = BeautifulSoup(bookmarksHtml, 'html.parser')

# extract <a> tags (games) within certain folders (["Gry na PS5  ", "indyki"]) and from these folders exclude bookmarks in other folders (["notReleased"])
target_text = ["Gry na PS5  ", "indyki"]
dl_tags = []

for h3_tag in soup.find_all('h3'):
    if h3_tag.text in target_text:
        dl_tag = h3_tag.find_next_sibling('dl')
        if dl_tag:
            dl_tags.append(dl_tag)

target_text = ["notReleased"]
dl_tags2 = []

for tag in dl_tags:
    for h3_tag in tag.find_all('h3'):
        if h3_tag.text not in target_text:
            dl_tag = h3_tag.find_next_sibling('dl')
            if dl_tag:
                dl_tags2.append(dl_tag)

a_tags = []

for tag in dl_tags2:
    a = tag.find_all("a")
    a_tags.extend([e.text for e in a])

In [3]:
games_dict = {}
titles = []
systems = []

# excluding only title and system for each game saved in Opera bookmarks as GryOnline.pl subsites
for tag_text in a_tags:
    parts = tag_text.split('|')
    if len(parts) == 2:
        extracted_part = parts[0].strip()
        matched_patterns = re.findall(r'PS\d+', extracted_part)
        modified_part = re.sub(r' (PS\d+|PC|XONE|XSX|Switch|iOS|X360)', '', extracted_part)
        modified_part2 = re.sub(r',+$', '', modified_part)
        parts2 = modified_part2.split(',')
        for p in parts2:
            titles.append(re.sub(r'^ ', '', p))
            systems.append(matched_patterns[0])

games_dict['title'] = titles
games_dict['system'] = systems


In [4]:
with open('Data/games.json', 'w', encoding='utf-8') as f:
    json.dump(games_dict, f)